In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler
from keras.constraints import maxnorm
from keras.datasets import cifar10

from matplotlib import pyplot as plt
%matplotlib inline

import cv2
import glob
import os
import re

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [18]:
# Reinstallise models 
img_size = 32

def cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, img_size, img_size), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu',border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(128, 3, 3, activation='relu',border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model


model2 = cnn_model()
modeladam = cnn_model()
modelada = cnn_model()

lr = 0.01

sgd = SGD(lr=lr, decay=1e-5, momentum=0.8, nesterov=True)

model2.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])

modeladam.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

modelada.compile(loss='categorical_crossentropy',
          optimizer='adadelta',
          metrics=['accuracy'])


#def lr_schedule(epoch):
    #return lr*(0.1**int(epoch/10))

In [19]:
# fitting the model
batch_size = 128
epochs = 50

model2.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs)


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 22s - loss: 1.8780 - acc: 0.3003 - val_loss: 1.5371 - val_acc: 0.4494
Epoch 2/50
50000/50000 [==============================] - 23s - loss: 1.4308 - acc: 0.4794 - val_loss: 1.2568 - val_acc: 0.5427
Epoch 3/50
50000/50000 [==============================] - 23s - loss: 1.2230 - acc: 0.5620 - val_loss: 1.2185 - val_acc: 0.5593
Epoch 4/50
50000/50000 [==============================] - 23s - loss: 1.0596 - acc: 0.6221 - val_loss: 1.0042 - val_acc: 0.6445
Epoch 5/50
50000/50000 [==============================] - 23s - loss: 0.9307 - acc: 0.6717 - val_loss: 0.8666 - val_acc: 0.6911
Epoch 6/50
50000/50000 [==============================] - 23s - loss: 0.8314 - acc: 0.7057 - val_loss: 0.7681 - val_acc: 0.7305
Epoch 7/50
50000/50000 [==============================] - 23s - loss: 0.7498 - acc: 0.7364 - val_loss: 0.7275 - val_acc: 0.7441
Epoch 8/50
50000/50000 [==============================

In [21]:
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.41%


In [15]:
## data augmentation


datagen1 = ImageDataGenerator(featurewise_center=False, 
                              featurewise_std_normalization=False,
                             #zca_whitening=True,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              zoom_range=0.2,
                              shear_range=0.1,
                              rotation_range=10.,
                              horizontal_flip=True,
                              vertical_flip=True)


datagen2 = ImageDataGenerator(featurewise_center=True,
                              featurewise_std_normalization=True,
                              #zca_whitening=True,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              zoom_range=0.2,
                              shear_range=0.1,
                              rotation_range=10.,
                              #horizontal_flip=True
                              )

datagen3 = ImageDataGenerator(featurewise_center=False,
                              featurewise_std_normalization=False,
                              zca_whitening=True)

datagen4 = ImageDataGenerator(zca_whitening=True)

datagen1.fit(X_train)

In [16]:
model2 = cnn_model()

lr = 0.01

sgd = SGD(lr=lr, decay=1e-5, momentum=0.9, nesterov=False)

model2.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])

nb_epoch =30
batch_size = 128
model2.fit_generator(datagen1.flow(X_train, y_train, batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_test, y_test))

Epoch 1/30
50000/50000 [==============================] - 15s - loss: 2.1475 - acc: 0.1914 - val_loss: 1.9643 - val_acc: 0.2706
Epoch 2/30
50000/50000 [==============================] - 14s - loss: 1.9189 - acc: 0.2858 - val_loss: 1.7399 - val_acc: 0.3717
Epoch 3/30
50000/50000 [==============================] - 14s - loss: 1.7610 - acc: 0.3480 - val_loss: 1.6027 - val_acc: 0.4083
Epoch 4/30
50000/50000 [==============================] - 14s - loss: 1.6465 - acc: 0.3986 - val_loss: 1.4756 - val_acc: 0.4547
Epoch 5/30
50000/50000 [==============================] - 14s - loss: 1.5612 - acc: 0.4320 - val_loss: 1.4690 - val_acc: 0.4668
Epoch 6/30
50000/50000 [==============================] - 14s - loss: 1.4810 - acc: 0.4603 - val_loss: 1.3196 - val_acc: 0.5203
Epoch 7/30
50000/50000 [==============================] - 14s - loss: 1.4323 - acc: 0.4769 - val_loss: 1.3170 - val_acc: 0.5216
Epoch 8/30
50000/50000 [==============================] - 14s - loss: 1.3784 - acc: 0.5001 - val_loss: 1

In [20]:
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.41%
